In [ ]:
include("OperatorAlgebra.jl")
include("Krylov.jl")

In [10]:
Id=unit(MajoranaOperator)
O1=Id*maj(4,5)
O2=O1*O1
println(O1, "\n", O2)

χ4 χ5
- I


In [3]:
O1=cdag(1,2) * c(1,2)
O2=O1*c(2)

println(O1,"\n",O2)

-n1 n2
0


In [4]:
O1=px(3,4,5)*pz(1,2,5)
O2=py(3)*py(3)
println(O1,"\n",O2)

-iZ1 Z2 X3 X4 Y5
I


### Bootstrap Sanity Test
Building Hubbard model with different operators to compare the ground state.

In [3]:
function orb(i::Int, s::Int)
    return 2i + s
end
function FHubbard(L::Int, t::Number, U::Number)
    T = UInt64   # UInt8 for L≤4, UInt16 for 5..7, else wider
    H = zero(FermionOperator{T})

    # Hopping: -t * Σ_{i,s} ( c†_{i,s} c_{i+1,s} + c†_{i+1,s} c_{i,s} ), Periodic
    for i in 0:L-1
        ip1 = (i + 1) % L
        for s in (0, 1)
            j = orb(i, s)
            jp = orb(ip1, s)
            H -= t * (cdag(T, j) * c(T, jp) + cdag(T, jp) * c(T, j))
        end
    end

    # On-site interaction: U * Σ_i n_{i,↑} n_{i,↓}
    for i in 0:L-1
        jdn = orb(i, 0)
        jup = orb(i, 1)

        H += U * (n(T, jdn) * n(T, jup))
    end
    return H
end


FHubbard (generic function with 1 method)

In [5]:
mc(i::Int, s::Int)  = (maj(4*i + 2*s) + 1im*maj(4*i + 2*s + 1)) / 2
mcd(i::Int, s::Int) = (maj(4*i + 2*s) - 1im*maj(4*i + 2*s + 1)) / 2
mn(i::Int, s::Int)  = mcd(i,s) * mc(i,s)

function MHubbard(L::Int, t::Number, U::Number)
    H = zero(MajoranaOperator{UInt64})
    for i in 0:L-1
        ip1 = (i + 1) % L
        for s in (0, 1)
            H -= t * (mcd(i, s) * mc(ip1, s) + mcd(ip1, s) * mc(i, s))
        end
    end
    for i in 0:L-1
        H += U * (mn(i, 0) * mn(i, 1))
    end
    return H
end


MHubbard (generic function with 1 method)

In [ ]:

@inline function JW_String(j::Int)::PauliOperator{UInt64}
    j <= 0 && return unit(PauliOperator{UInt64})
    zmask = (UInt64(1) << UInt64(j)) - UInt64(1)   
    return PauliOperator{UInt64}(Dict(PKey(UInt64(0), zmask) => 1.0 + 0im))
end

# Raising/lowering on a single site
@inline σplus(j::Int)  = (px(j) + (0+1im)*py(j)) / 2     # (X + iY)/2
@inline σminus(j::Int) = (px(j) - (0+1im)*py(j)) / 2     # (X - iY)/2


c(j::Int) = JW_String(j) * σplus(j)     
cd(j::Int)     = JW_String(j) * σminus(j)   

# Number operator construction
n_op(j::Int)  = cd(j) * c(j)              
n_op2(j::Int) = (unit(PauliOperator{UInt64}) - pz(j)) / 2    





function PHubbard(L::Int, t::Number, U::Number)
    H = zero(PauliOperator{UInt64})  

    for i in 0:L-1
        ip1 = (i + 1) % L                 
        for s in (0, 1)
            H -= t * (cd(2*i + s) * c(2*ip1+ s) + cd(2*ip1 + s) * c(2*i+s))
        end
    end

    for i in 0:L-1
        H += U * (n_op(2*i+ 0) * n_op2(2i+ 1))
    end

    return H
end



PHubbard (generic function with 1 method)

In [7]:
Hamiltonian=PHubbard(6,1,1)
rho = qboot(Hamiltonian, show_progress = true, max_power = 30, closure_depth = 24, enable_reorth=false)

println("Ground state energy: ", inner(Hamiltonian, rho))

>>> Building H-generated monomial space
closure depth 1: frontier=1, total=1
closure depth 2: frontier=42, total=43
closure depth 3: frontier=807, total=850
closure depth 4: frontier=9244, total=10094
closure depth 5: frontier=68547, total=78641
closure depth 6: frontier=331338, total=409979
closure depth 7: frontier=994581, total=1404560
closure depth 8: frontier=1624872, total=3029432
closure depth 9: frontier=1042248, total=4071680
closure depth 10: frontier=122504, total=4194184
closure depth 11: frontier=120, total=4194304
H-generated space: 4194304 monomials (depth=11)
    k   terms   weight      norm
 0:      1    1.000 1.000e+00
alpha= 1.500000e+00  norm= 2.669270e+00
 1:     42    1.000 2.669e+00
alpha= 1.578947e+00  norm= 3.450832e+00
 2:    753    1.000 3.451e+00
alpha= 1.579074e+00  norm= 3.864685e+00
 3:   7213    1.000 3.865e+00
alpha= 1.538660e+00  norm= 4.085505e+00
 4:  42976    1.000 4.086e+00
alpha= 1.489958e+00  norm= 4.182948e+00
 5: 172018    1.000 4.183e+00
alpha

In [8]:
Hamiltonian=MHubbard(6,1,1)
rho = qboot(Hamiltonian, show_progress = false, max_power = 30, closure_depth = 24, enable_reorth=false,reorth_first=0)

println("Ground state energy: ", inner(Hamiltonian, rho))


Ground state energy: -6.601158295113091 + 0.0im


In [4]:
Hamiltonian=FHubbard(6,1,1)
rho = qboot(Hamiltonian, show_progress = false, max_power = 30, closure_depth = 24, 
 enable_reorth=false,reorth_first=0)

println("Ground state energy: ", inner(Hamiltonian, rho))


Ground state energy: -6.601158300721084 + 0.0im



Exact Diagonalization gives ground truth E_0 = -6.601158293375125.

This shows that even with 30 krylov basis, the bootstrap approach converges to E_0 up to 1e-7 accuracy providing strong lower bound.
However, if Krylov subspace is spanned by fewer basis thus the optimization is performed in limited subspace, the ground energy might be higher than its ED value. 